# Siemens Energy Coding Challenge - Task 1 - 2021 - Sam MacIntyre

This jupyter notebook contains the code related to task 1, the jupyter notebook follows the structure below:

- Task outline
- Environment and version control
- Problem approach
- Data and library loading
- Data exploration and summary statistics
- Temporal alignment
- NAN treatment
- Moving average predictions
- Reproduce plots
- Conclusions and discussion


## Task outline

Time series data is often coming in different granularities and different frequencies. More often than not, we try to forecast or predict a timeseries value based on the historical observations of this time series as well as its causal dependency with another time series.
In this task you should use the file task1_60min_singleindex_filtered.csv to write a data processing pipeline that is able to:

- Temporally align the timeseries signals (please refer to the plots folder for a visual representation);
- Deal with the NAN values within the data, using methods up to your decision;
- Generate moving average hourly predictions up to 7 days into the future;
- Reproduce the plots provided after replacing the missing values and generating the predictions.

If you don't have sufficient computer power to use the whole dataset in the predictors generation part, feel free to use a subset of the data.

## Environment and version control

A temporary Github repository will be used to employ version control and protect against loss of code. Furthermore, a standard Python3 venv has been set up to ensure a clean and reproducible coding environment.

## Import libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

## Load data

In [24]:
df_ts = pd.read_csv('data/Task 1/task1_60min_singleindex_filtered.csv', encoding = 'latin-1')

/Users/sammacintyre/Documents/Mitiga/airport-contam/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Data exploration

In [25]:
# Check head of data to ensure proper reading
df_ts.head()

,utc_timestamp,cet_cest_timestamp,SE_load_actual_entsoe_power_statistics,SE_load_actual_entsoe_transparency,SE_load_actual_tso,SE_load_forecast_entsoe_transparency,SE_price_day_ahead,SE_solar_generation_actual,SE_wind_generation_actual,SE_wind_onshore_generation_actual,...,SE_2_price_day_ahead,SE_2_wind_onshore_generation_actual,SE_3_load_actual_entsoe_transparency,SE_3_load_forecast_entsoe_transparency,SE_3_price_day_ahead,SE_3_wind_onshore_generation_actual,SE_4_load_actual_entsoe_transparency,SE_4_load_forecast_entsoe_transparency,SE_4_price_day_ahead,SE_4_wind_onshore_generation_actual
0,2004-12-31T23:00:00Z,2005-01-01T00:00:00+0100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2005-01-01T00:00:00Z,2005-01-01T01:00:00+0100,NaN,NaN,-15991.34,NaN,NaN,NaN,3.22,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2005-01-01T01:00:00Z,2005-01-01T02:00:00+0100,NaN,NaN,-15673.12,NaN,NaN,NaN,2.84,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2005-01-01T02:00:00Z,2005-01-01T03:00:00+0100,NaN,NaN,-15431.81,NaN,NaN,NaN,2.57,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2005-01-01T03:00:00Z,2005-01-01T04:00:00+0100,NaN,NaN,-15326.48,NaN,NaN,NaN,3.12,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Check column types
df_ts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125593 entries, 0 to 125592
Data columns (total 26 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   utc_timestamp                           125593 non-null  object 
 1   cet_cest_timestamp                      125593 non-null  object 
 2   SE_load_actual_entsoe_power_statistics  78887 non-null   object 
 3   SE_load_actual_entsoe_transparency      37847 non-null   float64
 4   SE_load_actual_tso                      122711 non-null  float64
 5   SE_load_forecast_entsoe_transparency    37798 non-null   float64
 6   SE_price_day_ahead                      51112 non-null   float64
 7   SE_solar_generation_actual              74538 non-null   float64
 8   SE_wind_generation_actual               122711 non-null  float64
 9   SE_wind_onshore_generation_actual       38039 non-null   float64
 10  SE_1_load_actual_entsoe_transparency    3829

In [27]:
df_ts.describe()

,SE_load_actual_entsoe_transparency,SE_load_actual_tso,SE_load_forecast_entsoe_transparency,SE_price_day_ahead,SE_solar_generation_actual,SE_wind_generation_actual,SE_wind_onshore_generation_actual,SE_1_load_actual_entsoe_transparency,SE_1_load_forecast_entsoe_transparency,SE_1_price_day_ahead,...,SE_2_load_forecast_entsoe_transparency,SE_2_wind_onshore_generation_actual,SE_3_load_actual_entsoe_transparency,SE_3_load_forecast_entsoe_transparency,SE_3_price_day_ahead,SE_3_wind_onshore_generation_actual,SE_4_load_actual_entsoe_transparency,SE_4_load_forecast_entsoe_transparency,SE_4_price_day_ahead,SE_4_wind_onshore_generation_actual
count,37847.000000,122711.000000,37798.000000,51112.000000,74538.000000,122711.000000,38039.000000,38298.000000,38183.000000,38327.000000,...,38183.000000,38039.000000,38298.000000,38183.000000,65054.000000,38039.000000,38298.000000,38183.000000,65054.000000,38038.000000
mean,15836.095595,-15537.601393,15943.467908,45.395782,2021.479640,905.552023,1934.743737,1123.080879,1122.206389,32.297254,...,1952.388041,626.273745,10034.524623,10077.834156,33.509687,679.074095,2794.820095,2814.558038,34.488575,456.365634
std,3408.919933,3463.643985,3394.720625,25.475342,5695.203685,1016.791749,1163.672737,208.813328,188.363007,13.593987,...,371.275125,495.372174,2214.650738,2202.861492,13.802876,477.930163,668.545650,671.062383,13.977284,346.393558
min,8619.000000,-26265.307690,8911.000000,0.000000,0.000000,0.000000,18.000000,368.000000,26.560000,0.320000,...,26.560000,0.000000,5212.000000,26.560000,0.320000,2.000000,436.000000,26.560000,0.320000,1.000000
25%,13177.000000,-17966.510639,13272.000000,34.650000,0.010000,142.840000,997.000000,965.000000,968.000000,24.160000,...,1671.000000,212.000000,8298.250000,8347.000000,26.200000,285.000000,2283.000000,2295.000000,26.540000,171.000000
50%,15484.000000,-15183.600000,15622.500000,43.190000,0.064862,519.030000,1748.000000,1106.000000,1110.000000,30.200000,...,1905.000000,493.000000,9825.000000,9895.000000,32.060000,578.000000,2744.000000,2761.000000,32.590000,362.000000
75%,18231.000000,-12816.263240,18350.750000,54.020000,2.893642,1333.287513,2670.000000,1263.000000,1265.000000,40.130000,...,2214.000000,973.000000,11601.000000,11647.000000,39.717500,977.000000,3263.000000,3288.000000,40.940000,680.000000
max,26714.000000,15006.000000,26416.000000,1400.110000,26731.623710,5874.000968,6250.000000,3965.000000,3272.000000,737.000000,...,3568.000000,9345.000000,17677.000000,16957.000000,1331.000000,2503.000000,4921.000000,5014.000000,602.000000,1475.000000


Some initial comments from inspection of data:
- Two timestamps present, one in UTC time and the other in CET
- Both timestamps **not** in datetime format
- 23 individual timeseries present, 2 of which not in floating point format
- Timeseries are of different lengths and have different number of NANs


#### Treating timestamps

In [28]:
# Convert timestamps
df_ts['utc_timestamp'] = pd.to_datetime(df_ts['utc_timestamp'])

In [29]:
df_ts['cet_cest_timestamp'] = pd.to_datetime(df_ts['cet_cest_timestamp'])

**Comments**:

Three timestamps had to treated manually as they had errors:

1. timestamp contained #
2. í special character in timestamp
3. 3019 instead of 2019 present in timestamp

In [30]:
# Check timestamps
df_ts.head()

,utc_timestamp,cet_cest_timestamp,SE_load_actual_entsoe_power_statistics,SE_load_actual_entsoe_transparency,SE_load_actual_tso,SE_load_forecast_entsoe_transparency,SE_price_day_ahead,SE_solar_generation_actual,SE_wind_generation_actual,SE_wind_onshore_generation_actual,...,SE_2_price_day_ahead,SE_2_wind_onshore_generation_actual,SE_3_load_actual_entsoe_transparency,SE_3_load_forecast_entsoe_transparency,SE_3_price_day_ahead,SE_3_wind_onshore_generation_actual,SE_4_load_actual_entsoe_transparency,SE_4_load_forecast_entsoe_transparency,SE_4_price_day_ahead,SE_4_wind_onshore_generation_actual
0,2004-12-31 23:00:00+00:00,2005-01-01 00:00:00+01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2005-01-01 00:00:00+00:00,2005-01-01 01:00:00+01:00,NaN,NaN,-15991.34,NaN,NaN,NaN,3.22,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2005-01-01 01:00:00+00:00,2005-01-01 02:00:00+01:00,NaN,NaN,-15673.12,NaN,NaN,NaN,2.84,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2005-01-01 02:00:00+00:00,2005-01-01 03:00:00+01:00,NaN,NaN,-15431.81,NaN,NaN,NaN,2.57,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2005-01-01 03:00:00+00:00,2005-01-01 04:00:00+01:00,NaN,NaN,-15326.48,NaN,NaN,NaN,3.12,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Plot timeseries
